# Pipeline 1

In [1]:
from pipeline import Falcon2Linker, SerialAnnotator, T5Converter
import time
from pprint import pprint

main_virtuoso_noreranking


In [2]:
linker = Falcon2Linker()
annotator = SerialAnnotator()
converter = T5Converter()

In [3]:
utterance = "Who is the wife of Barack Obama"

In [4]:
linked = linker.link(utterance)
# rules = [1,2,3,4,5,8,9,10,12,13,14]
# linked = process_text_E_R(utterance, rules)
pprint("Linked")
pprint(linked)

pprint("Annotated")
annotated = annotator.annotate(**linked)
pprint(annotated)

pprint("Converted")
converted = converter.preprocess_inputs(**annotated)
pprint(converted)

recieved utterance
0 : ['Who is the wife of Barack Obama']


['Who is the wife of Barack Obama', [['<http://www.wikidata.org/entity/P26>', 'wife'], ['<http://www.wikidata.org/entity/P2848>', 'wife'], ['<http://www.wikidata.org/entity/P140>', 'wife'], ['<http://www.wikidata.org/entity/P451>', 'wife'], ['<http://www.wikidata.org/entity/P600>', 'wife']], [['<http://www.wikidata.org/entity/Q76>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q649593>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4808526>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4858106>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q643049>', 'Barack obama']], 0, 0, 0, 0]
'Linked'
{'ents': [{'id': 'Q76',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q76'},
          {'id': 'Q649593',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q649593'},
          {'id': 'Q4808526',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q4808526'},
          {'id': 'Q4858106',
  

{'fragments': ['[DEF]',
               'wd:',
               'Q76 Barack',
               '[DEF]',
               'wd:',
               'Q649593 Barack Hussein Obama, Sr.',
               '[DEF]',
               'wd:',
               'Q4808526 Obama assassination plot',
               '[DEF]',
               'wd:',
               'Q4858106 Barack Obama Academy',
               '[DEF]',
               'wd:',
               'Q643049 family of Barack Obama',
               '[DEF]',
               'wdt:',
               'P26 partner',
               '[DEF]',
               'wdt:',
               'P2848 wifi',
               '[DEF]',
               'wdt:',
               'P140 religion',
               '[DEF]',
               'wdt:',
               'P451 common-law spouse',
               '[DEF]',
               'wdt:',
               'P600 Wine AppDB-ID'],
 'utterance': 'Who is the wife of Barack Obama'}
'Converted'
('Who is the wife of Barack Obama <extra_id_59> <extra_id_53> Q76 Barack '

In [5]:
import re

def retrieve_gold_links(masked_wikisparql):
  fragments = masked_wikisparql.upper().strip().split(' ')
  ents = []
  rels = []
  result_dict = {
    'ents': ents,
    'rels': rels,
  }
  for fragment in fragments:
    id_match = re.search('[QP][1-9]+',fragment)
    if id_match:
      id_raw = id_match.group(0)
      if id_raw[0] == 'Q':
        prefix = 'wdt:'
        uri = 'http://www.wikidata.org/entity/' + id_raw
        target_arr = ents
      elif id_raw[0] == 'P':
        prefix = 'wd:'
        uri =  'http://www.wikidata.org/prop/direct/' + id_raw
        target_arr = rels
      else:
        raise ValueError('ID does not start with P or Q')
      match_dict = {
        'id' : id_raw,
        'prefix' : prefix,
        'uri' : uri,
      }
      target_arr.append(match_dict)
  return result_dict

In [6]:
def pipe(utterance, wikisparql):
    linked = linker.link(utterance)
    annotated = annotator.annotate(**linked)
    converted = converter.preprocess(**annotated, wikisparql=wikisparql)
    return linked, annotated, converted

In [7]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = linker.link_batch(utterances)

  assert len(linked) == len(wikisparqls)

  for i, link in enumerate(linked):
    gold_linked = retrieve_gold_links(wikisparqls[i])
    new_ents=link['ents']
    old_ents_set = set(x['id'] for x in link['ents'])
    for gold_ent in gold_linked['ents']:
        if gold_ent['id'] not in old_ents_set:
            new_ents.append(gold_ent)
    link['rels'] = []
    assert len(linked[i]['rels']) == 0

  link_batch_end = time.time()
  s = time.time()
  annotated = annotator.batch_annotate(linked)
  e = time.time()
  assert len(linked) == len(annotated)
  annotator_time = e - s
  converter_time = 0
  for i, single_annotated in enumerate(annotated):
    # annotated = annotator.annotate(**single_linked)
    # try:
    #   annotator_time += e - s
    # except Exception as e:
    #   print("[Annotator Error]:", e)
    #   continue
    s = time.time()
    try:
      converted = converter.preprocess(**single_annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([linked[i], annotated[i], converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [8]:
from pathlib import Path
import pandas as pd
import json

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"
data2_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "test_KG_4211.json"

In [9]:
with open(data_path) as f:
  data_dict = json.load(f)

with open(data2_path) as f:
  data_dict.extend(json.load(f))

count = 0

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  if paraphrased_question == None:
    raise Exception
  if question == None:
    count += 1
    # print(len(paraphrased_question) > 2)
    # pprint(item)
  if item["sparql_wikidata"] == None:
    raise Exception
  

  if len(paraphrased_question) > 2000:
    paraphrased_question = question
  # item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 and paraphrased_question != [] else question
  item["new_question"] = question

print("Missing question count:", count)

df = pd.DataFrame.from_dict(data_dict)
df = df.dropna(axis=0, subset=['new_question'])
df = df[~df.question.isin(['n/a', None, 'none', 'None'])]
n_entries = len(df)
print("Number of rows:", n_entries)

Missing question count: 71


Number of rows: 28247


In [10]:
assert(len(df[df.question.isna()]) == 0)

In [11]:
for data in df[0:5].iterrows():
    print(data[1].new_question)

What periodical literature does Delta Air Lines use as a moutpiece?
Who is the child of Ranavalona I's husband?
Is it true Jeff_Bridges occupation Lane Chandler and photographer ?
What is the pre-requisite of phase matter of Galinstan?
Which is the operating income for Qantas?


In [12]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = df[:]
total_len = len(truncated_data)
last = 0
cutoff = 0
for i, data in enumerate(truncated_data.iterrows()):
    data = data[1]
    if i < cutoff:
        continue
    # print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if (question and answer) and len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 50) == 0 or i == total_len - 1:
        print("[Pipeline1]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(f"../t5-for-sparql/falcon_links/5ents-gold_0rels/link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))

[Pipeline1]: Linking 0-49
1 : ['What periodical literature does Delta Air Lines use as a moutpiece?']
1 : ['Who is the child of Ranavalona Is husband?']
1 : ['Is it true Jeff_Bridges occupation Lane Chandler and photographer ?']
1 : ['What is the pre-requisite of phase matter of Galinstan?']
1 : ['Which is the operating income for Qantas?']
1 : ['which cola starts with the letter p']
1 : ['Is the right ascension of malin 1 less than 15.1398?']
1 : ['What is the complete list of records released by Jerry Lee Lewis?']
1 : ['What is Mary Lou Rettons International Olympic Committee athlete ID.']
1 : ['Who won the prize at the sequel of the 1885 Wimbledon Championships- Gentlemens Singles?']
1 : ['Is it true that the carbon footprint of the iPhone X Max is 106?']
1 : ['Tell me the female beauty pageant that operates in all countries and contains the word model in its name?']
1 : ['Did Lindsey Vonn participate in the FIS Alpine World Ski Championships in 1999 and 2019?']


11 : ['On 0-0-2007, what is the populaton of Spokane?']
 : ['When position did Angela Merkel hold on November 10, 1994?']
1 : ['What essential medicine is needed to treat leprosy?']
1 : ['Which is HanCinema person ID for Zhang Ziyi?']
1 : ['Did Judi Densch receive a Tony Award for best direction of the play Premium Imperiale?']
1 : ['What is the boiling point of pressure copper as 4703.0?']
1 : ['After what is Marathon named and what is the current record?']
1 : ['Who Sleepwalking succeeded in playing Sleepwalking?']
1 : ['What did Clifton Chenier get nominated for in 1983-0-0?']
1 : ['Which is the wear for Ambrose Burnside?']
1 : ['Which Class IB flammable liquid has the least lower flammable limit?']
1 : ['What is the material used and approved by Mojito?']
1 : ['Could you summarize Koreas history of this topic?']
1 : ['Which is {landscape of} of {Virgin of the rocks}, which has {birth city} is {Tzippori} ?']
1 : ['Is Kevin Costner owner of Fielders Stadium?']
1 : ['Which is the YouT

11 : ['How many dimensions have a Captain America?']
1 : ['What award did Danila Kozlovsky receive in 2017?']
 : ['What is a skill that starts with the letter s.']


2 : ['Who is the student that coined the Euler-Lagrange equation?']


3 : ['Which website does Twitch own?']
4 : ['Is the minimal lethal dose of the benzene equal to 170000?']


...

2059 : ['With a population of 434516, what is the capital of Majorca?']


2060 : ['Who is the father of Nefertiti?']
2061 : ['Is Uruguays minimum age for compulsory education the number 6?']
2062 : ['What is the earliest date for John the Baptists death in the year 30?']


2063 : ['Tell me freeway network whose name has the word system in it.']
2064 : ['Which is the Model Manual ID for Cindy Crawford?']


2065 : ['What is the U.S. National Geodetic Survey ID of Mount Mitchell?']
2066 : ['When was Niels Bohr  awarded the Nobel Prize in Physics? How much was he awarded?']


2067 : ['Which is the Scoresway soccer person ID for Gareth Bale?']
2068 : ['What are the writing system which start with the letter W']
2069 : ['What silent short film depicts Richard III of England?']
